In [1]:
from pulp import *
import pandas as pd
import numpy as np
import os

In [2]:
pwd()

'/Users/kunaal/Georgia Tech/Bootcamp'

In [3]:
os.chdir('/Users/kunaal/Georgia Tech/Fall 2019/Analytics Modeling/HW11/hw11/')

In [4]:
diet = pd.read_excel('Data 15.2/diet.xls', nrows=64)

diet_large = pd.read_excel('Data 15.2/diet_large.xls', skiprows = 1)

In [5]:
diet.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [6]:
diet_large.head()

,Long_Desc,Protein,"Carbohydrate, by difference",Energy,Water,Energy.1,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
0,"Butter, salted",0.85,0.06,717,15.87,3000.0,24,0.02,2,24,...,0.034,0.042,0.11,0.003,3,0.17,7,215.0,NaN,51.368
1,"Butter, whipped, with salt",0.85,0.06,717,15.87,2999.0,24,0.16,2,23,...,0.034,0.042,0.11,0.003,3,0.13,7,219.0,NaN,50.489
2,"Butter oil, anhydrous",0.28,0,876,0.24,3665.0,4,0,0,3,...,0.005,0.003,0.01,0.001,0,0.01,8.6,256.0,NaN,61.924
3,"Cheese, blue",21.4,2.34,353,42.41,1477.0,528,0.31,23,387,...,0.382,1.016,1.729,0.166,36,1.22,2.4,75.0,NaN,18.669
4,"Cheese, brick",23.24,2.79,371,41.11,1552.0,674,0.43,24,451,...,0.351,0.118,0.288,0.065,20,1.26,2.5,94.0,NaN,18.764


In [7]:
constraints = pd.read_excel('Data 15.2/diet.xls', skiprows=66, header=None).iloc[:,2:]

In [8]:
constraints

,2,3,4,5,6,7,8,9,10,11,12,13
0,Minimum daily intake,1500,30,20,800,130,125,60,1000,400,700,10
1,Maximum daily intake,2500,240,70,2000,450,250,100,10000,5000,1500,40


In [9]:
constraints.columns = diet.columns[2:]

In [10]:
constraints

,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Minimum daily intake,1500,30,20,800,130,125,60,1000,400,700,10
1,Maximum daily intake,2500,240,70,2000,450,250,100,10000,5000,1500,40


In [11]:
prob = pulp.LpProblem("Diet Problem", LpMinimize)

In [12]:
prob

Diet Problem:
MINIMIZE
None
VARIABLES

In [13]:
food_items = diet['Foods']

In [14]:
food_items

0          Frozen Broccoli
1              Carrots,Raw
2              Celery, Raw
3              Frozen Corn
4      Lettuce,Iceberg,Raw
              ...         
59         Neweng Clamchwd
60             Tomato Soup
61    New E Clamchwd,W/Mlk
62    Crm Mshrm Soup,W/Mlk
63    Beanbacn Soup,W/Watr
Name: Foods, Length: 64, dtype: object

In [15]:
var = LpVariable.dicts("Var", food_items, lowBound=0, cat='Continuous')

In [16]:
costs = dict(zip(food_items, diet['Price/ Serving']))

In [17]:
prob += lpSum([costs[i]*var[i] for i in food_items])

In [18]:
cols = diet.columns[3:]

In [19]:
for c in cols:
    prob += lpSum([diet[c][i]*var[food_items[i]] for i in range(len(var))]) >= constraints[c][0]
    prob += lpSum([diet[c][i]*var[food_items[i]] for i in range(len(var))]) <= constraints[c][1]

In [20]:
var

{'Frozen Broccoli': Var_Frozen_Broccoli,
 'Carrots,Raw': Var_Carrots,Raw,
 'Celery, Raw': Var_Celery,_Raw,
 'Frozen Corn': Var_Frozen_Corn,
 'Lettuce,Iceberg,Raw': Var_Lettuce,Iceberg,Raw,
 'Peppers, Sweet, Raw': Var_Peppers,_Sweet,_Raw,
 'Potatoes, Baked': Var_Potatoes,_Baked,
 'Tofu': Var_Tofu,
 'Roasted Chicken': Var_Roasted_Chicken,
 'Spaghetti W/ Sauce': Var_Spaghetti_W__Sauce,
 'Tomato,Red,Ripe,Raw': Var_Tomato,Red,Ripe,Raw,
 'Apple,Raw,W/Skin': Var_Apple,Raw,W_Skin,
 'Banana': Var_Banana,
 'Grapes': Var_Grapes,
 'Kiwifruit,Raw,Fresh': Var_Kiwifruit,Raw,Fresh,
 'Oranges': Var_Oranges,
 'Bagels': Var_Bagels,
 'Wheat Bread': Var_Wheat_Bread,
 'White Bread': Var_White_Bread,
 'Oatmeal Cookies': Var_Oatmeal_Cookies,
 'Apple Pie': Var_Apple_Pie,
 'Chocolate Chip Cookies': Var_Chocolate_Chip_Cookies,
 'Butter,Regular': Var_Butter,Regular,
 'Cheddar Cheese': Var_Cheddar_Cheese,
 '3.3% Fat,Whole Milk': Var_3.3%_Fat,Whole_Milk,
 '2% Lowfat Milk': Var_2%_Lowfat_Milk,
 'Skim Milk': Var_Skim

In [21]:
costs

{'Frozen Broccoli': 0.16,
 'Carrots,Raw': 0.07,
 'Celery, Raw': 0.04,
 'Frozen Corn': 0.18,
 'Lettuce,Iceberg,Raw': 0.02,
 'Peppers, Sweet, Raw': 0.53,
 'Potatoes, Baked': 0.06,
 'Tofu': 0.31,
 'Roasted Chicken': 0.84,
 'Spaghetti W/ Sauce': 0.78,
 'Tomato,Red,Ripe,Raw': 0.27,
 'Apple,Raw,W/Skin': 0.24,
 'Banana': 0.15,
 'Grapes': 0.32,
 'Kiwifruit,Raw,Fresh': 0.49,
 'Oranges': 0.15,
 'Bagels': 0.16,
 'Wheat Bread': 0.05,
 'White Bread': 0.06,
 'Oatmeal Cookies': 0.09,
 'Apple Pie': 0.16,
 'Chocolate Chip Cookies': 0.03,
 'Butter,Regular': 0.05,
 'Cheddar Cheese': 0.25,
 '3.3% Fat,Whole Milk': 0.16,
 '2% Lowfat Milk': 0.23,
 'Skim Milk': 0.13,
 'Poached Eggs': 0.08,
 'Scrambled Eggs': 0.11,
 'Bologna,Turkey': 0.15,
 'Frankfurter, Beef': 0.27,
 'Ham,Sliced,Extralean': 0.33,
 'Kielbasa,Prk': 0.15,
 "Cap'N Crunch": 0.31,
 'Cheerios': 0.28,
 "Corn Flks, Kellogg'S": 0.28,
 "Raisin Brn, Kellg'S": 0.34,
 'Rice Krispies': 0.32,
 'Special K': 0.38,
 'Oatmeal': 0.82,
 'Malt-O-Meal,Choc': 0.52,
 

In [22]:
prob.solve()

1

In [23]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [24]:
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Var_Celery,_Raw = 52.64371
Var_Frozen_Broccoli = 0.25960653
Var_Lettuce,Iceberg,Raw = 63.988506
Var_Oranges = 2.2929389
Var_Poached_Eggs = 0.14184397
Var_Popcorn,Air_Popped = 13.869322


In [25]:
obj = value(prob.objective)
print("The total cost of this balanced diet is: ${}".format(round(obj,2)))

The total cost of this balanced diet is: $4.34


In [26]:
selected = LpVariable.dicts("Selected", food_items, lowBound=0, upBound=1, cat="Binary")

In [27]:
for i in food_items:
    prob += var[i] >= 0.1*selected[i]
    prob += var[i] <= 1000001*selected[i]

In [28]:
prob += selected['Celery, Raw'] + selected['Frozen Broccoli'] <= 1 

In [29]:
meat_items = [
       'Bologna,Turkey', 'Frankfurter, Beef','Ham,Sliced,Extralean',
       'Hamburger W/Toppings', 'Kielbasa,Prk',
       'Pizza W/Pepperoni', 'Poached Eggs',
       'Pork', 'Roasted Chicken', 'Sardines in Oil',
       'Scrambled Eggs',
       'Splt Pea&Hamsoup', 'Vegetbeef Soup',
       'White Tuna in Water', 'Chicknoodl Soup']

In [30]:
prob += lpSum([selected[i] for i in meat_items]) >= 3

In [31]:
prob.solve()

1

In [32]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [33]:
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Selected_Celery,_Raw = 1.0
Selected_Kielbasa,Prk = 1.0
Selected_Lettuce,Iceberg,Raw = 1.0
Selected_Oranges = 1.0
Selected_Peanut_Butter = 1.0
Selected_Poached_Eggs = 1.0
Selected_Popcorn,Air_Popped = 1.0
Selected_Scrambled_Eggs = 1.0
Var_Celery,_Raw = 42.399358
Var_Kielbasa,Prk = 0.1
Var_Lettuce,Iceberg,Raw = 82.802586
Var_Oranges = 3.0771841
Var_Peanut_Butter = 1.9429716
Var_Poached_Eggs = 0.1
Var_Popcorn,Air_Popped = 13.223294
Var_Scrambled_Eggs = 0.1


In [34]:
obj = value(prob.objective)
print("The total cost of this balanced diet is: ${}".format(round(obj,2)))

The total cost of this balanced diet is: $4.51
